<a href="https://colab.research.google.com/github/redjules/sentiment-analysis/blob/main/Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. Install and Import Dependencies**



In [1]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.7 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

##**2. Instantiate Model**

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

##**3. Encode and Calculate Sentiment**

In [13]:
tokens = tokenizer.encode('Meh, it was ok', return_tensors='pt')

In [8]:
#tokens[0]

tensor([  101,   151, 39487, 10163, 10372,   117, 35925, 10563, 10103, 43060,
          102])

In [9]:
#tokenizer.decode(tokens[0])

'[CLS] i hated this, absolutely the worst [SEP]'

In [14]:
result = model(tokens)

In [15]:
result.logits

tensor([[-0.7896,  1.6718,  2.7634, -0.2691, -2.8158]],
       grad_fn=<AddmmBackward0>)

In [16]:
int(torch.argmax(result.logits))+1

3

##**4. Collect Reviews**

In [19]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [24]:
reviews[0]

'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!'

##**5. Load Reviews into DataFrame and Score**

In [25]:
import numpy as np
import pandas as pd

In [26]:
df = pd.DataFrame(np.array(reviews), 
                   columns=['review'])

In [28]:
df['review'].iloc[0]

'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!'

In [35]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [36]:
sentiment_score(df['review'].iloc[1])




5

In [37]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [38]:
df

,review,sentiment
0,The food and service here was really good. It...,5
1,Visiting from Texas and decided to give this r...,5
2,"Great atmosphere, attentive service, solid mar...",3
3,Don't come here expecting legit Mexican food b...,3
4,Out of all the restaurants that I tried in Syd...,5
5,We came here on a Thursday night @ 5pm and by ...,4
6,The food is fresh and tasty. The scallop cevi...,4
7,Have been here twice and have absolutely loved...,5
8,I was pleasantly surprised at what a great job...,5
9,If you're looking for a quiet little romantic ...,2


In [34]:
df['review'].iloc[3]

"Don't come here expecting legit Mexican food but a modern twist on some staples. Loud party area, fun drinks and friendly staff make this a hip meeting area for large groups. Drinks were better than the food. They stuff the families toward the back but lack any amenities (no changing table) except a high chair. Service started off friendly but it took a while to get someone to take our order and then they forgot our dish which came out cold when we asked for it. Then we had to flag someone down to pay the bill. The watermelon salad was tasty but not complex, tossed with a few cucumbers and pistachios. The corn lollipops with spicy mayo sauce were probably the best dish. The beef empanadas were cold and average though the salsa was an interesting pickled onion. Definitely skip the octopus dish- it was dry, over cooked and lacked flavor though the presentation looked promising. It came with squares of polenta and dallops of potato mash with a dramatic squid ink streak across the dish bu